# Why is there some spatial banding in candidates?

In [1]:
%matplotlib inline
from pylab import *
import numpy as np
from craco.search_pipeline import location2pix, get_grid_lut_from_plan, grid_candidates
import craco



In [2]:
fits='/data/craco/ban115/test_data/frb_d0_t0_a1_sninf_lm00/frb_d0_t0_a1_sninf_lm00.fits'
xclbin='/data/craco/ban115/builds/binary_container_1_10688715.xclbin'

In [3]:
!search_pipeline --uv $fits  -x $xclbin -T 3  --input-scale 4 -C  cand4.txt --dump-mainbufs 1 --dump-fdmt-hist-buf 1 --dump-candidates 1 --dump-uvdata 1

INFO:craco.search_pipeline:Values=Namespace(boxcar_weight='sum', cand_file='cand4.txt', cell=None, device=0, dump_boxcar_hist_buf=None, dump_candidates=1, dump_fdmt_hist_buf=1, dump_mainbufs=1, dump_uvdata=1, fdmt_scale=1.0, fft_scale=10.0, input_scale=4.0, nblocks=1, nbox=8, ncin=32, ndm=2, ndout=186, npix=256, nt=256, nuvmax=8192, nuvwide=8, os='2.1,2.1', run_fdmt=True, run_image=True, show=False, show_candidate_grid=None, threshold=3.0, uv='/data/craco/ban115/test_data/frb_d0_t0_a1_sninf_lm00/frb_d0_t0_a1_sninf_lm00.fits', verbose=False, wait=False, xclbin='/data/craco/ban115/builds/binary_container_1_10688715.xclbin')
INFO:craco.search_pipeline:fdmt_tunable_c32:fdmt_tunable_c32_1
INFO:craco.search_pipeline:krnl_ddgrid_reader_4cu:krnl_ddgrid_reader_4cu_1
INFO:craco.search_pipeline:krnl_grid_4cu:krnl_grid_4cu_1
INFO:craco.search_pipeline:krnl_grid_4cu:krnl_grid_4cu_2
INFO:craco.search_pipeline:krnl_grid_4cu:krnl_grid_4cu_3
INFO:craco.search_pipeline:krnl_grid_4cu:krnl_grid_4cu_4
INFO

In [4]:
!ls -lrth

total 11G
-rw-rw-r-- 1 ban115 ban115  57K Dec 13 13:32 'Check FDMT.ipynb'
-rw-rw-r-- 1 ban115 ban115  11K Feb 21 18:37  candidates.txt
-rw-rw-r-- 1 ban115 ban115  11K Feb 21 19:14  cand2.txt
-rw-rw-r-- 1 ban115 ban115   40 Feb 21 19:15  cand3.txt
-rw-rw-r-- 1 ban115 ban115  73K Feb 21 21:07  uv_data.uvgrid.txt
-rw-rw-r-- 1 ban115 ban115 382K Feb 21 21:07  uv_data.uvgrid.split.txt
-rw-rw-r-- 1 ban115 ban115  86K Feb 21 21:07  uv_data.gridlut.upper.txt
-rw-rw-r-- 1 ban115 ban115  86K Feb 21 21:07  uv_data.gridlut.lower.txt
-rw-rw-r-- 1 ban115 ban115 325K Feb 21 21:07  uv_data.padlut.upper.txt
-rw-rw-r-- 1 ban115 ban115 8.1K Feb 21 21:07  uv_data.doshift.upper.txt
-rw-rw-r-- 1 ban115 ban115 322K Feb 21 21:07  uv_data.padlut.lower.txt
-rw-rw-r-- 1 ban115 ban115 8.1K Feb 21 21:07  uv_data.doshift.lower.txt
-rw-rw-r-- 1 ban115 ban115 165M Feb 22 09:18  uv_data_iblk0.npy
-rw-rw-r-- 1 ban115 ban115  192 Feb 22 09:18  candidates_iblk0.npy
-rw-rw-r-- 1 ban115 ban115 1.3G Feb 22 09:18  mainbuf_af

In [5]:
d = np.load('candidates_iblk0.npy')
print("got", len(d), 'candidates')

#mainbuf_clear = np.load('../src/craco/mainbuf_after_clearing.npy')
mainbuf_run = np.load('mainbuf_after_run_iblk0_ib0.npy')
uv_data = np.load('uv_data_iblk0.npy')

npix=256

got 0 candidates


In [6]:
#d = d[d['dm'] == 0]

In [7]:
location2pixv = np.vectorize(location2pix)

In [8]:
xpix, ypix = location2pixv(d['loc_2dfft'], npix)

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

In [ ]:
img = grid_candidates(d, 'snr')    
imshow(img[:,:], aspect='auto', vmin=0, vmax=5)
gcf().set_size_inches(8,8)

In [ ]:
img = grid_candidates(d, 'count')    
imshow(np.sqrt(img[:,:]), aspect='auto', vmin=0, vmax=5)
gcf().set_size_inches(8,8)

In [ ]:
plot(d['time'])
xlabel('Detection')
ylabel('time')

In [ ]:
plot(d['dm'])
xlabel('Detection')
ylabel('idm')

In [ ]:
plot(d['snr'])
xlabel('Detection')
ylabel('snr')

In [ ]:
scatter(d['time'],d['dm'])
ylabel('dm')
xlabel('time')

In [ ]:
print(mainbuf_run.shape)

In [ ]:
pltd = mainbuf_run[0,:,0,:,0,0]
imshow(pltd, aspect='auto', origin='lower')
xlabel('Time')
ylabel('dm')
print(np.max(pltd))

In [ ]:
plot(mainbuf_run[0,0,0,:,0,0])

In [ ]:
print(uv_data.shape)

In [ ]:
sum(uv_data)

In [ ]:
print('real', sum(uv_data[:,0,:,:,0]), 'imag', sum(uv_data[:,0,:,:,1]))

In [ ]:
#Load the whole buffer as one go into memory. Painful but possible.
main_nuv = mainbuf_run.shape[0]
mainbuf_shape =list(mainbuf_run.shape[:])
nbuf = 8
mainbuf_shape[0] *= nbuf
mainbuf = np.zeros(mainbuf_shape, dtype=np.int16)
print(mainbuf_shape)
for b in range(8):
    start = b*main_nuv
    end = (b+1)*main_nuv
    d =  np.load(f'mainbuf_after_run_iblk0_ib{b}.npy')
    print(start, end, b, main_nuv, d.shape)

    mainbuf[start:end, ...] = d
    # (101, 186, 11, 256, 8, 2)



In [ ]:
mainbuf.shape

In [ ]:
# Check times > 0 for DM = 0 is all zero
assert np.all(mainbuf[:,0,0,1:,:,:] == 0), 'FDMT of DM0 for t>1 should be 0'


In [ ]:
imshow(mainbuf[:,0,0,1:,0,0])

In [ ]:
# check sum of UV data at DM = 0 is the sum of DM 0 values after FDMT
print(uv_data.shape)

print('Sum of uv data at t=0, real', sum(uv_data[:,0,:,:,0]))
# check UV data is DM =
assert sum(uv_data) == sum(uv_data[:,0,:,:,0]), 'UV data contians some power not at t=0'

# find DM 0 values for all UVs
print('sum of FDMT values for D=0, T=0', sum(mainbuf[:,0,:,0,:,0]))
# check sum of DM0 values for all UVs equals sum of T=0 for all input channels
assert sum(uv_data[:,0,:,:,0]) == sum(mainbuf[:,0,:,0,:,0]), 'Sum of DM0 for all UVs doesnt equal sum of all channels at t=0'


In [ ]:
print(mainbuf.shape)
for tblk in range(11):
    print(tblk, mainbuf[:,0,tblk,:,:, 0].sum())

In [ ]:
!python -V
!pip install -e /data/craco/ban115/realtime_pipeline_master/python/craco_pybind11/

In [ ]:
from imp import reload
import sys
print(sys.path)
import craco_pybind11
import craco_pybind11.ddgrid_reader
import craco_pybind11.grid
reload(craco_pybind11)

#reload(craco_pybind11.ddgrid_reader)



In [ ]:
from craft.craco_plan import PipelinePlan
import craco
import craft.craco_plan
from craft import uvfits
ndm = 1
tblk = 0
NUVWIDE = 8
OUTPUT_NT = 2
OUTPUT_NUV = 2
#nuvrest = nuv // NUVWIDE
#nuvrest = plan.nuvrest
ncu = 4
do_load_lut=1
f = uvfits.open(fits)
plan = PipelinePlan(f, craft.craco_plan.get_parser().parse_args(['--ndm',str(ndm)]))
nchunk_time = plan.nchunk_time

nparallel_uvin, nparallel_uvout, h_nparallel_uvout, lut = get_grid_lut_from_plan(plan)
nuvrest = nparallel_uvin*2//8
print(nparallel_uvin, nparallel_uvout, h_nparallel_uvout)
print(lut)

In [ ]:
#outputs = np.zeros((ncu, ndm, nchunk_time, nuvrest, 4, 2, dtype=np.int16) for n in range(ncu)]
outputs = np.zeros((ncu, ndm, nchunk_time, nuvrest, NUVWIDE//OUTPUT_NUV, OUTPUT_NT, OUTPUT_NUV, 2), dtype=np.int16)
ddreader_lut = np.zeros(1024+plan.nuvrest,dtype=np.uint32)

In [ ]:
#     # (101, 186, 11, 256, 8, 2)
mainbuf[:] = 0
t = 4
mainbuf[:,0,0,t,:,0] = 1

craco_pybind11.ddgrid_reader.krnl_ddgrid_reader_4cu(mainbuf, 
                                     ndm, 
                                     tblk,
                                     nchunk_time, 
                                     nuvrest, 
                                     ddreader_lut, 
                                     do_load_lut, 
                                     outputs[0], 
                                     outputs[1], 
                                     outputs[2], 
                                     outputs[3])

In [ ]:
reordered = np.transpose(outputs, [1, 2,0,5, 3,4,6, 7])
#assert reordered.shape == (ndm, nchunk_time, ncu, OUTPUT_NT, nuvrest, NUVWIDE, 2)
reordered = reordered.reshape(ndm, ncu*nchunk_time*OUTPUT_NT, nuvrest*NUVWIDE, 2)


In [ ]:
reordered.shape
imshow(reordered[0,:,:,0], aspect='auto', origin='lower', interpolation='none')
print(abs(reordered).sum(), mainbuf.sum())

In [ ]:
#outputs = np.zeros((ncu, ndm, nchunk_time, nuvrest, NUVWIDE, OUTPUT_NT, 2), dtype=np.int16)

print(outputs.sum())
print(outputs[1,0,0,:,:,1,0].sum())
for icu in range(outputs.shape[0]):
    print('cu=', icu, outputs[icu, ...].sum())

In [ ]:
plot(reordered[0,:,:,0].T);

In [ ]:
outputs[0].sum()

In [ ]:
load_luts=True
d_grid = np.zeros((ncu, ndm, nchunk_time, plan.npix, plan.npix, 2), dtype=np.int16)
print(d_grid.sum(), d_grid.shape)
print(nparallel_uvin, ndm, nchunk_time)
print(outputs[icu].shape, outputs[icu].size)

icu=2
print(outputs[icu].sum())
craco_pybind11.grid.krnl_grid_4cu(ndm,
                     nchunk_time,
                     nparallel_uvin,
                     nparallel_uvout,
                     h_nparallel_uvout,
                     load_luts,
                     lut,
                     outputs[icu],
                     d_grid[icu])
print(d_grid[icu].sum())
print(d_grid.shape)


In [ ]:
for icu in range(ncu):
    print(outputs[icu].sum(),d_grid[icu].sum())


In [ ]:
fig, ax = subplots(1,2)
ax[0].imshow(d_grid[2,0,0,:,:,0])
ax[1].imshow(d_grid[2,0,0,:,:,1])

In [ ]:
np.all(d_grid[:,0:,:,:,:] == 0)

In [ ]:
np.all(d_grid[:,1:,:,:,:] == 0)

In [ ]:
def image_fft(g, scale='none'):
    '''
    Do the complex-to-complex imaging FFT with the correct shifts and correct inverse thingy
    If g.shape = (Npix, Npix) then we assume the center of the UV plane is at
    Npix/2, Npix/2 = DC
    Noramlised by np.prod(img.shape) - which I think is the same as the Xilinx FFT
    
    :scale: 'none' or None for raw FFT output. 'prod' for np.prod(g.shape)

    '''
    # The old version was incorrect!
    #cimg = np.fft.fftshift(np.fft.ifft2(g)).astype(np.complex64)

    if scale == 'none':
        s = 1.0
    elif scale == 'prod':
        s = np.prod(g.shape)
    
    cimg = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(g)))/s
    return cimg


def dofft(grid):
    ndm, nchunk, npix, _, _ = grid.shape
    images = np.zeros((ndm, nchunk, npix, npix, 2), dtype=np.float32)
    for dm in range(ndm):
        for chunk in range(nchunk):
            g = np.zeros((npix, npix), dtype=np.complex64)
            g.real = grid[dm, chunk, :, :, 0]
            g.imag = grid[dm, chunk, :, :, 1]
            img = image_fft(g, scale='prod')
            images[dm, chunk, :, :, 0] = img.real
            images[dm, chunk, :, :, 1] = img.imag         
    
    return images

In [ ]:
images = dofft(d_grid[2,...])

In [ ]:
images.shape

In [ ]:
fig,ax = subplots(1,2)
islice = slice(120, 140)
ax[0].imshow(images[0,0,islice,islice,0])
ax[1].imshow(images[0,0,islice,islice,1])

print(images.max())
print(images[0,0,islice,islice,:].max(axis=0).max(axis=0))

In [ ]:
np.all(images[:,:,:,:,1:] == 0)

In [ ]:
# KB - TODO check ddgrid_reader and grid aggree on ordering within a FIFO word - I think it should be UTC (UV, Time, Complex) but 
# perhaps not. Can I check with just hte code?

In [ ]:
# the problem is with the gridding - it isn't hermetian enough
greal = d_grid[2,0,0,:,:,0]
gimag = d_grid[2,0,0,:,:,1]
